In [ ]:
import os
from data_gen import (
    Utils,
    load_from_jsonl,
    save_to_jsonl,
    step_1_prompt_gen,
    step_2_prompt_gen,
    step_3_prompt_gen,
    step_4_prompt_gen,
    step_5_prompt_gen,
    step_6_prompt_gen,
)

path = 'krx_본선/데이터생성/'
api_key = '<api_key>'
util = Utils(api_key, path)

# Step 2

In [ ]:
## ChatGPT를 통한 주제 수집

fin_topic_lst = '''기업의 매출과 순이익의 차이
간단한 기업 예산 계획 세우기
회사의 고정비용과 변동비용 구분'''.split('\n')

acc_topic_lst = '''기업의 매출과 순이익의 차이
간단한 기업 예산 계획 세우기
회사의 고정비용과 변동비용 구분'''.split('\n')

print(len(fin_topic_lst))
print(len(acc_topic_lst))

In [ ]:
## 데이터 생성 프롬프트 제작 / 생성 배치 실행
prompt_save_file_name = path + 'data/step_2_prompt.jsonl'

gpt_prompt_lst = step_2_prompt_gen(fin_topic_lst, mode='fin') + step_2_prompt_gen(acc_topic_lst, mode='acc')
save_to_jsonl(prompt_save_file_name, gpt_prompt_lst)
util.gpt_batch_request(gpt_prompt_lst)

In [ ]:
## 데이터 생성 현황 검색
util.batch_status()

In [ ]:
## 생성 결과 저장
result_save_file_name = path + 'data/step_2_result.jsonl'
util.gpt_result_file_save(result_save_file_name)

In [ ]:
## 데이터 전처리
prep_save_file_name = path + 'data/step_2_prep.jsonl'
step_2_result = load_from_jsonl(result_save_file_name)

data_lst = []
for i, data in enumerate(step_2_result):
    id = data['custom_id']
    context = data['response']['body']['choices'][0]['message']['content']
    data_lst.append({'id': id, 'context': context})
print(len(data_lst))
save_to_jsonl(prep_save_file_name, data_lst)

# Step 3

In [ ]:
## 데이터 생성 프롬프트 제작 / 생성 배치 실행
prompt_save_file_name = path + 'data/step_3_prompt.jsonl'
step_2_prep = load_from_jsonl(prep_save_file_name)

gpt_prompt_lst = step_3_prompt_gen(step_2_prep)
save_to_jsonl(prompt_save_file_name, gpt_prompt_lst)
util.gpt_batch_request(gpt_prompt_lst)

In [ ]:
## 데이터 생성 현황 검색
util.batch_status()

In [ ]:
## 생성 결과 저장
result_save_file_name = path + 'data/step_3_result.jsonl'
util.gpt_result_file_save(result_save_file_name)

In [ ]:
## 데이터 전처리
prep_save_file_name = path + 'data/step_3_prep.jsonl'
step_2_prep = load_from_jsonl(prep_save_file_name)
step_3_result = load_from_jsonl(result_save_file_name)

data_lst = []
for data_1, data_2 in zip(step_2_prep, step_3_result):
    id = data_1['id']
    context = data_1['context']
    policy_data = data_2['response']['body']['choices'][0]['message']['content']
    if len(policy_data.split('\n'))==2:
        description = policy_data.split('\n')[0].split(':')[1].strip()
        policy = eval(policy_data.split('\n')[1].split(':')[1].strip())
    else:
        description = policy_data.split(' 판별')[0].split(':')[1].strip()
        policy = eval(policy_data.split(' 판별')[1].split(':')[1].strip())
    if policy:
        data_lst.append({'id': id, 'context': context, 'description': description, 'policy': policy})
print(len(data_lst))
save_to_jsonl(prep_save_file_name, data_lst)

# Step 4

In [ ]:
## 데이터 생성 프롬프트 제작 / 생성 배치 실행
prompt_save_file_name = path + 'data/step_4_prompt.jsonl'
step_3_prep = load_from_jsonl(prep_save_file_name)

gpt_prompt_lst = step_4_prompt_gen(step_3_prep)
save_to_jsonl(prompt_save_file_name, gpt_prompt_lst)
util.gpt_batch_request(gpt_prompt_lst)

In [ ]:
## 데이터 생성 현황 검색
util.batch_status()

In [ ]:
## 생성 결과 저장
result_save_file_name = path + 'data/step_4_result.jsonl'
util.gpt_result_file_save(result_save_file_name)

In [ ]:
## 데이터 전처리
prep_save_file_name = path + 'data/step_4_prep.jsonl'
step_3_result = load_from_jsonl(result_save_file_name)

data_lst = []
for data in step_3_result:
    id = data['custom_id']
    quiz_context = data['response']['body']['choices'][0]['message']['content']
    if '문제 ' in quiz_context:
        quiz_lst = [x.strip() for x in re.split(r'문제\s\d\.', quiz_context.replace('---', '').replace('###', ''))[1:]]
    elif '\n\n---\n\n' in quiz_context:
        quiz_lst = [re.sub('\*\*문제\s\d\.\*\*\s', '', x).replace('---', '').strip() for x in quiz_context.split('\n\n---\n\n')]
    else:
        quiz_lst = [quiz_context]
    quiz_lst = [x.replace('\n\n', '\n').replace('**', '').strip() for x in quiz_lst]

    data_lst.append({'id': id[:-2], 'quiz_context': quiz_context, 'quiz_lst': quiz_lst})
print(len(data_lst))
aa = [len(x['quiz_lst']) for x in data_lst]
print(min(aa))
print(max(aa))

In [ ]:
def quiz_prep(quiz):
    question = quiz[:quiz.find('\nA.')].replace(':', '.').strip()
    if ('해설' not in quiz):
        return '', [''], '', '', False
    choice_lst = [x.strip() for x in quiz[quiz.find('\nA.')+1:quiz.find('\n해설')].split('\n') if re.search(r'^[ABCDEFGH]\.\s', x)]
    description = quiz[quiz.find('\n해설')+4:(quiz.find('\n해설')+4+quiz[quiz.find('\n해설')+4:].find('\n')) if '\n' in quiz[quiz.find('\n해설')+4:] else (-1)].replace('<br>', '').strip()
    if '\n정답' in quiz:
        target = quiz[quiz.find('\n정답')+3:quiz.find('\n해설')].strip()
    elif ' 따라서,' in quiz:
        target = quiz[quiz.find(' 따라서,')+5:]
    else:
        target = ' '

    if description[0]==':':
        description = description[1:].strip()
    if target[0]==':':
        target = target[1:].strip()
    if '정답이다' in target:
        target = target.strip()[0]
    description = description if description[-1]=='.' else description + '.'

    policy = False
    for choice in choice_lst:
        if target[:3] in choice:
            target = choice[0]
            policy = True
            break
    return question, choice_lst, description, target, policy

json_lst = []
id_lst = []
n = 0
for data in data_lst:
    id = data['id']
    quiz_lst = data['quiz_lst']
    if id not in id_lst:
        n = 0
    for quiz in quiz_lst:
        question, choice_lst, description, target, policy = quiz_prep(quiz)
        if any([x=='' for x in [question, description, target]]) or (not policy):
            continue
        json_lst.append({'id':'{}_{}'.format(id, n), 'question':question, 'choice_lst':'\n'.join(choice_lst), 'description':description, 'target':target, 'policy':policy})
        n += 1
    id_lst.append(id)

res_lst = []
for i, data in enumerate(json_lst):
    question = data['question']
    choice_lst = data['choice_lst']
    description = data['description']
    target = data['target']

    id = '{}'.format(data['id'])
    prompt = '''### 질문: {}
### 선택지:
{}

### 정답:'''.format(question, choice_lst)
    data_dict = {
        'id': id,
        'prompt': prompt,
        'target': description + '\n' + target
    }
    res_lst.append(data_dict)
print(len(res_lst))
save_to_jsonl(prep_save_file_name, data_dict)

# Step 5

In [ ]:
## 데이터 생성 프롬프트 제작 / 생성 배치 실행
prompt_save_file_name = path + 'data/step_4_prompt.jsonl'
step_4_prep = load_from_jsonl(prep_save_file_name)

gpt_prompt_lst = step_5_prompt_gen(step_4_prep)
save_to_jsonl(prompt_save_file_name, gpt_prompt_lst)
util.gpt_batch_request(gpt_prompt_lst)

In [ ]:
## 데이터 생성 현황 검색
util.batch_status()

In [ ]:
## 생성 결과 저장
result_save_file_name = path + 'data/step_5_result.jsonl'
util.gpt_result_file_save(result_save_file_name)

In [ ]:
## 데이터 전처리
prep_save_file_name = path + 'data/step_5_prep.jsonl'
step_4_prep = load_from_jsonl(prompt_save_file_name)
step_5_result = load_from_jsonl(result_save_file_name)
print(len(quiz_data))

def ch_policy_judge(prompt):
    word_lst = ['{}.'.format(x) for x in 'ABCDEFGH']
    choice_lst = prompt[prompt.find('### 선택지:\n')+9:prompt.find('\n\n### 정답:')].split('\n')
    return all([c[:2]==w for c,w in zip(choice_lst, word_lst[:len(choice_lst)])])

data_lst = []
for data_1, data_2 in zip(step_4_prep, step_5_result):
    id = data_2['custom_id']
    prompt = data_1['prompt'].replace('### 질문 :', '### 질문:')
    target = data_1['target']
    description = target[:-2]+'.' if target[-3]!='.' else target[:-2]
    answer = target[-1]

    ch_policy = ch_policy_judge(prompt)
    policy = eval(data_2['response']['body']['choices'][0]['message']['content'])

    if ch_policy and policy:
        data_lst.append({'id': id, 'prompt': prompt, 'description': description, 'answer': answer})
print(len(data_lst))
save_to_jsonl(prep_save_file_name, data_lst)

# Step 6

In [ ]:
## 데이터 생성 프롬프트 제작 / 생성 배치 실행
prompt_save_file_name = path + 'data/step_5_prompt.jsonl'
step_5_prep = load_from_jsonl(prep_save_file_name)

gpt_prompt_lst = step_6_prompt_gen(step_5_prep)
save_to_jsonl(prompt_save_file_name, gpt_prompt_lst)
util.gpt_batch_request(gpt_prompt_lst)

In [ ]:
## 데이터 생성 현황 검색
util.batch_status()

In [ ]:
## 생성 결과 저장
result_save_file_name = path + 'data/step_6_result.jsonl'
util.gpt_result_file_save(result_save_file_name)

In [ ]:
## 데이터 전처리
prep_save_file_name = path + 'data/step_5_prep.jsonl'
step_5_prep = load_from_jsonl(prompt_save_file_name)
step_6_result = load_from_jsonl(result_save_file_name)

def desc_split(context):
    step_1 = context[context.find('1) '):context.find('\n2) ')].strip()
    step_2 = context[context.find('2) '):context.find('\n3) ')].strip()
    step_3 = context[context.find('3) '):].strip()

    return step_1, step_2, step_3

def desc_split_2(context):
    step_1 = context[context.find('문제 이해'):context.find('문제 풀이')].strip()
    step_2 = context[context.find('문제 풀이'):context.find('최종 정답')].strip()
    step_3 = context[context.find('최종 정답'):].strip()
    step_1 = re.sub(r'\n\d\){,1}\.{,1}', '', step_1).strip()
    step_2 = re.sub(r'\n\d\.', '', step_2).strip()
    return '1) '+step_1, '2) '+step_2, '3) '+step_3

data_lst = []
for data_1, data_2 in zip(step_5_prep, step_6_result):
    id = data_2['custom_id']
    quiz = data_1['prompt'].split('\n\n### 정답')[0]
    context = data_2['response']['body']['choices'][0]['message']['content']

    data_lst.append({'id': id, 'quiz': quiz, 'context': context})
print(len(data_lst))

res = []
for data in data_lst:
    id = data['id']
    quiz = data['quiz']
    context = data['context'].replace('**', '').replace('###', '')
    context = re.sub(r'\n\s{0,5}\n', '\n', context)
    step_1, step_2, step_3 = desc_split(context)
    if any([len(x)<5 for x in [step_1, step_2, step_3]]):
        step_1, step_2, step_3 = desc_split_2(context)
    res.append([id, quiz, (step_1, step_2, step_3)])

In [ ]:
def step_1_prep(step_1):
    if '이해:' in step_1:
        split_text = step_1.split('이해:')
        title = split_text[0] + '이해:'
        context = split_text[1].strip()
    elif '\n' in step_1:
        split_text = step_1.split('\n')
        title = split_text[0].strip()
        if title[-1]!=':':
            title = title + ':'
        context = '\n'.join(split_text[1:]).strip()
    else:
        return step_1, False
    if title != '1) 문제 이해:':
        return step_1, False
    context = context.replace('\n#', '')
    return '\n'.join([title, context]), True

def step_2_prep(step_2):
    step_2 = step_2.replace('\n\n', '\n')
    split_text = step_2.split('문제 풀이')
    title = split_text[0] + '문제 풀이:'
    context = '문제 풀이'.join(split_text[1:]).strip()
    if context[0] == ':':
        context = context[1:].strip()
    imsi = []
    for text in context.split('\n'):
        if text.strip()[0] != '-':
            text = '- ' + text.strip()
        imsi.append(text.strip())
    re_lst = [
        ('\\(', '('),
        ('\\times', '*'),
        ('\\%', '/'),
        ('\\)', ')'),
        ('\\$', '$'),
    ]
    context = '\n'.join(imsi)
    for word_tup in re_lst:
        context = context.replace(word_tup[0], word_tup[1])
    if re.search(r"\\frac\{([\d,]+)\}\{([\d,]+)\}", context):
        search = re.search(r"\\frac\{([\d,]+)\}\{([\d,]+)\}", context)
        num_1 = search.group(1)
        num_2 = search.group(2)
        frac_text = f'{num_1}/{num_2}'
        context = re.sub(r"\\frac\{([\d,]+)\}\{([\d,]+)\}", frac_text, context)

    if len(context.split('\n'))==1:
        context = '\n'.join([x if x[0]!='-' else '- '+x for x in context.split('. ')])

    if title != '2) 문제 풀이:':
        return step_2, False
    if not re.search(r'^[\u3131-\u3163\uac00-\ud7a3a-zA-Z0-9\s+\-*/=(){}[\]^%<>×.,‧·‘’▲\$∑Σ∏μσδ√“”τ±÷αβΔ_!?|~`"\'&:;]+$', context):
        return context, False
    context = context.replace('\n#', '')
    return '\n'.join([title, context]), True

def step_3_prep(step_3):
    step_3 = step_3.replace('\n\n', '\n')
    split_text = step_3.split('최종 정답')
    context = split_text[1].replace(',', '').replace('.', '').replace('"', '').strip()
    if context[0] == ':':
        context = context[1:].strip()
    if len(context) != 1:
        return context, False
    return context, True

In [ ]:
def prompt_step(id, quiz, step_1, step_2, step_3):
    prompt_lst = []
    # step_1
    target = '\n\n'.join([step_1, step_2])
    data_dict = {
        'id': '{}_step{}'.format(id, 1),
        'prompt': quiz + '### 정답:',
        'target': target
    }
    prompt_lst.append(data_dict)
    # step_2
    data_dict = {
        'id': '{}_step{}'.format(id, 2),
        'prompt': quiz + '### 정답:' + target + '### 정답:',
        'target': step_3
    }
    prompt_lst.append(data_dict)
    return prompt_lst

train_prompt_lst = []
for id, quiz, (step_1, step_2, step_3) in res:
    try:
        step_1_text, s1_policy = step_1_prep(step_1)
        step_2_text, s2_policy = step_2_prep(step_2)
        step_3_text, s3_policy = step_3_prep(step_3)
    except:
        print(id)
        continue
    if all([s1_policy, s2_policy, s3_policy]) and (len(step_2_text.split('\n'))<13):
        train_prompt_lst.extend(prompt_step(id, quiz, step_1_text, step_2_text, step_3_text))
print(len(train_prompt_lst))
save_to_jsonl(path + 'train_data/mcqa_vf_data_2.jsonl', train_prompt_lst)